# Demonstrate Online Recommendation
This notebook demonstrates how recommendations can be queried dynamically in real time based on the KNN Similarity relationships in the graph. The queries in this notebook can be directly transferred to a REST API to serve live applications

In [71]:
import pandas as pd
import configparser
from datetime import datetime, timedelta

In [72]:
pd.set_option('display.max_rows', 12)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 0)

In [73]:
config = configparser.RawConfigParser()
config.read('neo4j.ini')
HOST = config['NEO4J']['HOST']
USERNAME = config['NEO4J']['USERNAME']
PASSWORD = config['NEO4J']['PASSWORD']

### Set Customer Example and Max Date

In [74]:
CUSTOMER_ID = '116ab76d8e060972ebbe162ab80e06022d8980db63d11c24f952916ad29e8508'
MAX_DATE = datetime(year=2020, month=9, day=22)

### Connect to Graph Data Science (GDS)

In [75]:
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to your setup
gds = GraphDataScience(HOST, auth=(USERNAME, PASSWORD), aura_ds=True)

## Example Customer Purchase History

In [76]:
%%time
gds.run_cypher('''
    MATCH(c:Customer {customerId:$customerId})-[r:PURCHASED]->(a) WHERE r.transactionDate > date($cutOffDate)
        RETURN a.articleId AS articleId, 
        a.productName AS productName,
        a.productTypeName AS productTypeName,
        a.detailDesc AS detailDesc
''', params = {'cutOffDate':str(MAX_DATE  - timedelta(days=42))[:10], 'customerId':CUSTOMER_ID})

CPU times: user 27.9 ms, sys: 0 ns, total: 27.9 ms
Wall time: 1.65 s


,articleId,productName,productTypeName,detailDesc
0,0923569002,Scout,Top,"Short dress in a soft rib knit containing some wool. Relaxed fit with dropped shoulders, long sleeves and slits in the sides. Ribbing around the neckline, cuffs and hem."
1,0923037003,Hongkong,Sweater,"Fitted jumper in a soft rib knit containing some wool, with a turtle neck and long sleeves with short slits at the cuffs."
2,0902528005,POPPY SHIRT DRESS,Dress,"Short, straight-cut shirt dress in a cotton weave with a collar, buttons down the front and a chest pocket. Dropped shoulders, long sleeves with buttoned cuffs, and a rounded hem. Slightly longer at the back."
3,0899122001,Shelf pile jacket,Jacket,"Straight-cut jacket in soft faux shearling with a hood, zip down the front, handwarmer pockets and pockets in the side seams. Gently dropped shoulders and long sleeves. Slightly longer at the back. Lining in a recycled polyester weave."
4,0923037001,Hongkong,Sweater,"Fitted jumper in a soft rib knit containing some wool, with a turtle neck and long sleeves with short slits at the cuffs."
...,...,...,...,...
11,0898684001,Sedona Skirt(1),Skirt,"Fitted, calf-length skirt in a soft, rib knit containing some wool. High waist with covered elastication, and a slit in one side. Unlined."
12,0896161001,Barbara,Cardigan,"Cardigan in a soft cable knit containing some wool. Boxy fit with dropped shoulders, long sleeves and ribbing at the cuffs and hem. Buttons down the front. The polyester content of the cardigan is recycled."
13,0685813042,PETAR SWEATSHIRT,Sweater,"Top in soft sweatshirt fabric. Slightly looser fit with dropped shoulders and ribbing around the neckline, cuffs and hem. Soft brushed inside."
14,0843465002,Jack Skinny Trousers PID,Trousers,"Suit trousers in a stretch weave with a concealed hook-and-eye fastener and zip fly. Diagonal side pockets, jetted back pockets, and an adjustable tab with buttons at the sides. Skinny Fit – a fit with slightly shorter legs that is close-fitting at the thighs, knees and ankles to create a completely fitted silhouette."


## Product Recommendation Without GDS - Pure Cypher
This is a great improvement over SQL but...
 - Returns many potential recommendations - can lack focus and be difficult to prioritize
 - Large queries - can be slow for some production use cases
 - Results set grows with size of data - can be difficult to scale for some use cases

In [82]:
%%time
gds.run_cypher('''
    MATCH(c:Customer {customerId:$customerId})-[r:PURCHASED]->(a0) WHERE r.transactionDate > date($cutOffDate)
    WITH c, a0
    MATCH(a0)<-[:PURCHASED]-(c0)-[r:PURCHASED]->(a)
    RETURN c.customerId AS customerId,
       count(DISTINCT a0) AS articlesPurchased,
       count(DISTINCT c0) AS CustomerAccountsTraversed,
       count(DISTINCT a) AS RecommendationCount
''', params = {'cutOffDate':str(MAX_DATE  - timedelta(days=42))[:10], 'customerId':CUSTOMER_ID})

CPU times: user 21.6 ms, sys: 3.92 ms, total: 25.5 ms
Wall time: 2.32 s


,customerId,articlesPurchased,CustomerAccountsTraversed,RecommendationCount
0,116ab76d8e060972ebbe162ab80e06022d8980db63d11c24f952916ad29e8508,15,5944,56637


In [78]:
#%%time
#gds.run_cypher('''
#    MATCH(c:Customer {customerId:$customerId})-[r:PURCHASED]->(a0) WHERE r.transactionDate > date($cutOffDate)
#    WITH a0
#    MATCH(a0)<-[:PURCHASED]-(c0)-[r:PURCHASED]->(a)
#        RETURN a.articleId AS articleId, 
#        a.productName AS productName,
#        a.productTypeName AS productTypeName,
#        a.detailDesc AS detailDesc
#''', params = {'cutOffDate':str(MAX_DATE  - timedelta(days=42))[:10], 'customerId':CUSTOMER_ID})

CPU times: user 34.1 s, sys: 715 ms, total: 34.9 s
Wall time: 2min 22s


,articleId,productName,productTypeName,detailDesc
0,0917447002,Magnolia PU quilt,Skirt,Short skirt in quilted imitation leather with a high waist and concealed zip in one side. Lined in taffeta made from recycled polyester.
1,0696785006,SCARLETT scarf scrunchie,Hair string,Elasticated scrunchie in woven fabric with long ties.
2,0920084003,Mull,Cardigan,"V-neck cardigan in a soft rib knit containing some wool. Relaxed fit with dropped shoulders, long sleeves and buttons down the front. The polyester content of the cardigan is recycled."
3,0763469009,Dores long hoodie(1),Sweater,"Oversized top in soft sweatshirt fabric made from a cotton blend with a lined, drawstring hood. Low dropped shoulders, concealed pockets in the side seams and ribbing at the cuffs and hem. Soft brushed inside."
4,0881942002,Fox,Blouse,"Fitted, smocked blouse in woven fabric with a frill-trimmed boat neck, long, voluminous puff sleeves with smocking and a frill trim at the cuffs, and a frill trim at the hem."
...,...,...,...,...
812862,0680979001,Taxi Taxi Dress,Dress,"Short, sleeveless dress in lace with a round neckline, concealed zip and hook-and-eye fastener at the back, and lining in a contrasting colour."
812863,0804732001,Durham joggers(1),Trousers,"Joggers in a soft, fine knit with a high, elasticated drawstring waist and tapered legs."
812864,0605970004,Linnea PJ (K),Pyjama set,"Pyjamas in soft, patterned jersey. Long-sleeved top with ribbing at the cuffs and hem. Bottoms with an elasticated waist and wide, gently tapered legs with ribbed hems."
812865,0656719005,Serpente HW slim trouser,Trousers,"Tailored trousers in a stretch weave with two press-studs at the waist and a zip fly. Zipped front pockets, fake welt back pockets and slim, straight legs."


## Product Recommendation Post GDS KNN Application
- Returns refined set of recommendations - personalized and relevant to user
- Smaller queries - Fast for enterprise use cases
- Results size remains constant & focused as data size increases - built for scale

In [88]:
%%time
gds.run_cypher('''
    MATCH(c:Customer {customerId:$customerId})-[r:PURCHASED]->(a0) WHERE r.transactionDate > date($cutOffDate)
    WITH c, a0
    MATCH(a0)-[s:CUSTOMERS_ALSO_PURCHASED]->(a)
    RETURN c.customerId AS customerId,
       count(DISTINCT a0) AS articlesPurchased,
       0 AS CustomerAccountsTraversed,
       count(DISTINCT a) AS RecommendationCount
''', params = {'cutOffDate':str(MAX_DATE  - timedelta(days=42))[:10], 'customerId':CUSTOMER_ID})

CPU times: user 3.5 ms, sys: 4.14 ms, total: 7.65 ms
Wall time: 279 ms


,customerId,articlesPurchased,CustomerAccountsTraversed,RecommendationCount
0,116ab76d8e060972ebbe162ab80e06022d8980db63d11c24f952916ad29e8508,14,0,118


In [89]:
%%time
gds.run_cypher('''
    MATCH(c:Customer {customerId:$customerId})-[r:PURCHASED]->(a0) WHERE r.transactionDate > date($cutOffDate)
    WITH a0
    MATCH(a0)-[s:CUSTOMERS_ALSO_PURCHASED]->(a)
    RETURN a.articleId AS articleId, 
        sum(s.score) AS aggScore, 
        a.productTypeName AS productType,
        a.productName AS name,
        a.percievedColorValueName AS color,
        a.detailDesc AS description
        ORDER BY aggScore DESC
''', params = {'cutOffDate':str(MAX_DATE - timedelta(days=42))[:10], 'customerId':CUSTOMER_ID})

CPU times: user 20.7 ms, sys: 181 µs, total: 20.9 ms
Wall time: 418 ms


,articleId,aggScore,productType,name,color,description
0,0582462006,2.339457,Slippers,Pierra sandal,Dusty Light,"Slides with a large bow at the front, imitation leather insoles and thermoplastic rubber (TPR) soles."
1,0860367001,2.326027,Top,Malin Cardigan,Medium Dusty,"Cardigan in a fine knit with a supersoft, brushed finish. V-neck, buttons down the front, long sleeves and ties at the hem."
2,0923037001,1.701222,Sweater,Hongkong,Dark,"Fitted jumper in a soft rib knit containing some wool, with a turtle neck and long sleeves with short slits at the cuffs."
3,0923037002,1.648601,Sweater,Hongkong,Dusty Light,"Fitted jumper in a soft rib knit containing some wool, with a turtle neck and long sleeves with short slits at the cuffs."
4,0757983002,1.640043,Shorts,Jossan Shorts,Dusty Light,"Short shorts in soft viscose twill with a high waist that is elasticated at the back, a zip fly and button, pleats at the front and pockets in the side seams."
...,...,...,...,...,...,...
113,0764045001,0.751596,Hoodie,Charli hood,Dusty Light,"Jacket in sweatshirt fabric with a lined drawstring hood, stand-up collar and zip down the front. Decorative seams and concealed pockets at the front, low dropped shoulders, long sleeves and ribbing at the cuffs and hem. Soft brushed inside."
114,0723201003,0.751226,Shirt,SOHO SHIRT,Medium,Wide V-neck blouse in a viscose weave with short sleeves and buttons down the front.
115,0607983001,0.750568,Skirt,Linus flounce skirt,Light,Tiered skirt in a striped cotton weave with smocking at the waist. Unlined.
116,0758170001,0.750367,Vest top,Orsino top,Dusty Light,Cropped top in crinkled viscose with crocheted shoulder straps and crocheted trims at the top and front. Buttons down the front and narrow elastication at the hem.


## Recommendations Based on Latest Purchases
The above recommendations traverse all purchases after the cutoff date. However, depending on use case, you may need to base recommendations on just the latest items the user purchased or interacted with. With the embeddings + KNN approach, you can use Cypher to customize the range of activity.
 
Below is an example of using just the last purchased item for recommendation. This produces more focused results.

In [90]:
%%time
# Last purchased item
gds.run_cypher('''
    MATCH(c:Customer {customerId:$customerId})-[r:PURCHASED]->()
    WITH c, max(r.transactionDate) AS maxTransactionDate
    MATCH(c:Customer {customerId:$customerId})-[r:PURCHASED]->(a) 
    WHERE r.transactionDate = maxTransactionDate
    RETURN a.articleId AS articleId, 
        a.productTypeName AS productType,
        a.productName AS name,
        a.percievedColorValueName AS color,
        a.detailDesc AS description
    ''', params={'customerId': CUSTOMER_ID})

CPU times: user 7.8 ms, sys: 0 ns, total: 7.8 ms
Wall time: 314 ms


,articleId,productType,name,color,description
0,0923569002,Top,Scout,Medium Dusty,"Short dress in a soft rib knit containing some wool. Relaxed fit with dropped shoulders, long sleeves and slits in the sides. Ribbing around the neckline, cuffs and hem."
1,0923037001,Sweater,Hongkong,Dark,"Fitted jumper in a soft rib knit containing some wool, with a turtle neck and long sleeves with short slits at the cuffs."
2,0898684001,Skirt,Sedona Skirt(1),Dark,"Fitted, calf-length skirt in a soft, rib knit containing some wool. High waist with covered elastication, and a slit in one side. Unlined."


In [91]:
%%time
# Recommendation Based on Last Purchased Items
gds.run_cypher('''
    MATCH(a0:Article {articleId:$articleId})-[s:CUSTOMERS_ALSO_PURCHASED]->(a)
    RETURN a.articleId AS articleId, 
        sum(s.score) AS aggScore, 
        a.productTypeName AS productType,
        a.productName AS name,
        a.percievedColorValueName AS color,
        a.detailDesc AS description
        ORDER BY aggScore DESC
''', params = {'articleId':'0923037001'})

CPU times: user 7.26 ms, sys: 0 ns, total: 7.26 ms
Wall time: 310 ms


,articleId,aggScore,productType,name,color,description
0,0898684001,0.872897,Skirt,Sedona Skirt(1),Dark,"Fitted, calf-length skirt in a soft, rib knit containing some wool. High waist with covered elastication, and a slit in one side. Unlined."
1,0757983002,0.830789,Shorts,Jossan Shorts,Dusty Light,"Short shorts in soft viscose twill with a high waist that is elasticated at the back, a zip fly and button, pleats at the front and pockets in the side seams."
2,0783636003,0.830642,Boots,Tyler boot,Dark,"Ankle boots in grained imitation leather with covered heels, elastic gores in the sides and a loop at the back. Satin linings, imitation leather insoles and smooth soles. Heel 6.5 cm."
3,0735550003,0.829120,Shorts,Freddy Shorts,Dusty Light,"Short, wide shorts in a viscose weave with a high, elasticated, frill-trimmed waist and side pockets."
4,0923037003,0.828325,Sweater,Hongkong,Dark,"Fitted jumper in a soft rib knit containing some wool, with a turtle neck and long sleeves with short slits at the cuffs."
5,0677561004,0.828176,Blouse,Love Dog,Light,"Long-sleeved blouse in a crêpe weave with a small stand-up collar, an opening with covered buttons at the back of the neck and wide cuffs with a covered button."
6,0853807003,0.826287,Leggings/Tights,Nudy Jazz Pants,Dark,"Jazz trousers in ribbed jersey with a slight sheen. High waist with concealed elastication, and straight-cut legs with flared hems."
7,0760463001,0.824709,Sneakers,Lindsey espadrille sneaker,Light,Trainers in canvas with lacing at the front and platform soles covered in braided jute. Canvas insoles and fluted soles. Platform approx. 4 cm.
8,0923037002,0.824658,Sweater,Hongkong,Dusty Light,"Fitted jumper in a soft rib knit containing some wool, with a turtle neck and long sleeves with short slits at the cuffs."
9,0720547003,0.824382,Dress,Parma Dress,Dusty Light,"Short dress in airy chiffon with a deep V-neck with pleats and covered buttons at the top. Gathers on the shoulders, long puff sleeves and cuffs with a covered button. Seam at the waist and above the hem. Partly lined."
